In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

In [2]:
dataDir = '/kaggle/input/mmds-cxr'
test_x = np.load(dataDir+'/test_images.npy')
y_true = np.load(dataDir+'/test_labels.npy')

test_x/=255.0

print('Test: {} , {}'.format(test_x.min(), test_x.max()))

Test: 0.0 , 1.0


In [3]:
class StochasticDepth(tf.keras.layers.Layer):
    """Stochastic Depth module.

    It performs batch-wise dropping rather than sample-wise. In libraries like
    `timm`, it's similar to `DropPath` layers that drops residual paths
    sample-wise.

    References:
      - https://github.com/rwightman/pytorch-image-models

    Args:
      drop_path_rate (float): Probability of dropping paths. Should be within
        [0, 1].

    Returns:
      Tensor either with the residual path dropped or kept.
    """

    def __init__(self, drop_path_rate, **kwargs):
        super().__init__(**kwargs)
        self.drop_path_rate = drop_path_rate

    def call(self, x, training=None):
        if training:
            keep_prob = 1 - self.drop_path_rate
            shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x

    def get_config(self):
        config = super().get_config()
        config.update({"drop_path_rate": self.drop_path_rate})
        return config

In [4]:
class LayerScale(tf.keras.layers.Layer):
    """Layer scale module.

    References:
      - https://arxiv.org/abs/2103.17239

    Args:
      init_values (float): Initial value for layer scale. Should be within
        [0, 1].
      projection_dim (int): Projection dimensionality.

    Returns:
      Tensor multiplied to the scale.
    """

    def __init__(self, init_values, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.init_values = init_values
        self.projection_dim = projection_dim

    def build(self, input_shape):
        self.gamma = tf.Variable(
            self.init_values * tf.ones((self.projection_dim,))
        )

    def call(self, x):
        return x * self.gamma

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "init_values": self.init_values,
                "projection_dim": self.projection_dim,
            }
        )
        return config

In [5]:
model = tf.keras.models.load_model('/kaggle/input/convnext-mmds-cxr/Model/convnext-best-model.h5', custom_objects={ "LayerScale": LayerScale, "StochasticDepth": StochasticDepth })

In [6]:
predictions = model.predict(x=test_x, batch_size=32)
y_pred = tf.keras.backend.argmax(predictions, axis=-1)

8/8 [==============================] - 470s 57s/step


In [7]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names=["Normal", "Pneumonia", "COVID-19"]))

              precision    recall  f1-score   support

      Normal       0.64      0.85      0.73       100
   Pneumonia       0.73      0.71      0.72       100
    COVID-19       0.00      0.00      0.00        31

    accuracy                           0.68       231
   macro avg       0.46      0.52      0.48       231
weighted avg       0.59      0.68      0.63       231

